In [1]:
import os
import re
import glob
from collections import OrderedDict

os.chdir('/home/ruidong/Documents/Research/share/Cox/miRNA/')

In [3]:
survFiles = sorted(glob.glob('*s.txt'))

In [4]:
for fl in survFiles:
    fh = open(fl[:-3]+'sig.txt', 'wt')
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[0] == 'coef':
                fh.write(line+'\n')
                
            elif float(lst[-2]) < 0.05:
                fh.write(line+'\n')
                
    fh.close()

In [5]:
GPL570 = OrderedDict()
with open('../GPL570-55999.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if line.startswith('#') or lst[0]=='ID':
            continue
        if len(lst) < 10:
            continue
            
        GPL570[lst[0]] = lst[10].split(' /// ')

In [7]:
survGenes = []

sigFiles = sorted(glob.glob('new.TCGA*sig.txt'))
for fl in sigFiles:
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[0] == 'coef':
                continue
                
            else:
                gene = lst[0].split('|')[0]
                if gene not in survGenes:
                    survGenes.append(gene)
len(survGenes)

37486

In [36]:
sigFiles = sorted(glob.glob('GEO*sig.txt'))
for fl in sigFiles:
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[0] == 'coef':
                continue
            
            elif lst[0] not in GPL570.keys():
                #print (lst[0])
                continue
            else:
                genes = GPL570[lst[0]]
                for gene in genes:
                    if gene not in survGenes:
                        survGenes.append(gene)
len(survGenes)

10859

In [59]:
samples = []
with open('methy_clinical.csv') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split(',')
        if lst[0] == 'ID':
            continue
        samples.append(lst[0]+'-01A')
        

In [67]:
probes = []

fh1 = open('methylation.clean.txt', 'wt')

with open('methylation.beta_value450_only.symbol.TCGA-COAD.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Probe':
            samIndex = [lst.index(sam) for sam in samples if sam in lst]
            fh1.write('\t'+'\t'.join([lst[i] for i in samIndex])+'\n')
        elif lst[1] not in survGenes:
            continue
        elif lst[0] in probes:
            continue
        else:
            probes.append(lst[0])
            fh1.write(lst[0]+'\t'+'\t'.join([lst[i] for i in samIndex])+'\n')
            
fh1.close()

In [69]:
survFiles = sorted(glob.glob('TCGA.Methy*.txt'))

for fl in survFiles:
    fh = open(fl[:-3]+'sig.txt', 'wt')
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[0] == 'coef':
                fh.write(line+'\n')
            elif lst[-2] == 'NA':
                continue
            elif float(lst[-2]) < 0.05:
                fh.write(line+'\n')
                
    fh.close()

In [8]:
methy450Anno = OrderedDict()

with open('../methylation450_annotation.clean.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        genes = set(lst[4].split(';'))
        methy450Anno[lst[0]] = genes
        
        


In [71]:
#############
## univariate coxph os ***
## multivariate coxph os ***
## univariate survdiff os <median, 0.3>
## multivariate survdiff os <median, 0.3>

## univariate coxph rfs ***
## multivariate coxph rfs ***
## univariate survdiff rfs <median, 0.3> ***
## multivariate survdiff rfs <median, 0.3> ***

1
2
3
5


In [106]:
sigFiles = sorted(glob.glob('*univariate.survdiff.rfs.sig.txt'))
geneSet1 = OrderedDict()
geneSet2 = OrderedDict()
ovlp = OrderedDict()

fh = open('Univariate.survdiff.median.os.overlap.txt', 'wt')
with open('GEO.Expr.univariate.survdiff.os.sig.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'coef':
            fh.write('Gene\tProbe\t'+line+'\n')
            continue
        
        if lst[0] in GPL570.keys():
            genes = GPL570[lst[0]]
            for gene in genes:
                geneSet1[gene] = gene+'\t'+line
                #print (gene)
                
with open('TCGA.Expr.univariate.survdiff.os.sig.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'coef':
            continue
        
        gene = lst[0].split('|')[0]
        if gene in geneSet1.keys():
            geneSet2[gene] = gene+'\t'+line
            
            
with open('TCGA.Methy.univariate.survdiff.os.sig.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'coef':
            continue
        
        elif lst[0] not in methy450Anno.keys():
            continue
            
        genes = methy450Anno[lst[0]]
        
        for gene in genes:
            if gene in geneSet2.keys():
                fh.write('\n'.join([geneSet1[gene],geneSet2[gene],gene+'\t'+line])+'\n\n')
fh.close()
            

In [89]:
GPL570['202345_s_at']

['FABP5']

In [92]:
methy450Anno['cg07701018']

{'FABP5', 'RP11-363E6.3'}

In [107]:
sigFiles = sorted(glob.glob('*multivariate.survdiff.rfs.sig.txt'))
geneSet1 = OrderedDict()
geneSet2 = OrderedDict()
ovlp = OrderedDict()

fh = open('Multivariate.survdiff.median.os.overlap.txt', 'wt')
with open('GEO.Expr.multivariate.survdiff.os.sig.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'coef':
            fh.write('Gene\tProbe\t'+line+'\n')
            continue
        
        if lst[0] in GPL570.keys():
            genes = GPL570[lst[0]]
            for gene in genes:
                geneSet1[gene] = gene+'\t'+line
                #print (gene)
                
with open('TCGA.Expr.multivariate.survdiff.os.sig.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'coef':
            continue
        
        gene = lst[0].split('|')[0]
        if gene in geneSet1.keys():
            geneSet2[gene] = gene+'\t'+line
            
            
with open('TCGA.Methy.multivariate.survdiff.os.sig.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'coef':
            continue
        
        elif lst[0] not in methy450Anno.keys():
            continue
            
        genes = methy450Anno[lst[0]]
        
        for gene in genes:
            if gene in geneSet2.keys():
                fh.write('\n'.join([geneSet1[gene],geneSet2[gene],gene+'\t'+line])+'\n\n')
fh.close()
            

In [110]:
uniGene = []
multiGene = []

with open('Univariate.coxph.os.overlap.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene' or lst[0]== '':
            #fh.write(line+'\n')
            continue
            
        if lst[0] not in uniGene:
            uniGene.append(lst[0])
            

with open('Multivariate.coxph.os.overlap.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene' or lst[0]== '':
            #fh.write(line+'\n')
            continue
            
        if lst[0] in uniGene and lst[0] not in multiGene:
            multiGene.append(lst[0])
            print (lst[0])
            

LMAN2
RAPGEF5
HIST4H4
PBK
OXSM
PDGFB
WHSC1
MARCH6
DNAH17
PPA2
DHX15
FDFT1
PRRX2
ESPNL
HSPA1L
SAMD11
RNF4
NEK9
RAB7A
PLEKHM3
TACC3
COQ2
SENP7
ASRGL1
KCP
ALDH3B1
ANKRD6
ATP5B


In [ ]:

with open('Univariate.coxph.os.overlap.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene':
            fh.write(line+'\n')
            continue
            
        if lst[0] not in uniGene:
            uniGene.append(lst[0])
            

with open('Multivariate.coxph.os.overlap.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene' or lst[0]== '':
            #fh.write(line+'\n')
            continue
            
        if lst[0] in uniGene and lst[0] not in multiGene:
            multiGene.append(lst[0])
            print (lst[0])

In [132]:
sigFiles = sorted(glob.glob('GEO*.sig.txt'))
sigFiles

for fl in sigFiles:
    fh = open(fl[:-3]+'symbol.txt', 'wt')
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[0] == 'coef':
                fh.write('Gene\tProbe\t'+line+'\n')
                continue
            elif lst[0] not in GPL570.keys():
                continue
            genes = GPL570[lst[0]]
            for gene in genes:
                fh.write(gene+'\t'+line+'\n')
                
    fh.close()


In [10]:
sigFiles = sorted(glob.glob('new.TCGA.Methy*.sig.txt'))
sigFiles

for fl in sigFiles:
    fh = open(fl[:-3]+'symbol.txt', 'wt')
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[0] == 'coef':
                fh.write('Gene\tProbe\t'+line+'\n')
                continue
            elif lst[0] not in methy450Anno.keys():
                continue
            genes = methy450Anno[lst[0]]
            for gene in genes:
                fh.write(gene+'\t'+line+'\n')
                
    fh.close()

In [11]:
sigFiles = sorted(glob.glob('new.TCGA.Expr*.sig.txt'))
sigFiles

for fl in sigFiles:
    fh = open(fl[:-3]+'symbol.txt', 'wt')
    with open(fl) as f:
        for line in f:
            line = line.rstrip()
            lst = line.split('\t')
            if lst[0] == 'coef':
                fh.write('Gene\tProbe\t'+line+'\n')
                continue
            else:
                gene = lst[0].split('|')[0]
                fh.write(gene+'\t'+line+'\n')
                
    fh.close()

In [15]:
geneSet1 = OrderedDict()
geneSet2 = OrderedDict()
geneSet3 = OrderedDict()
geneSet4 = OrderedDict()
geneSet5 = OrderedDict()
geneSet6 = OrderedDict()

     
fh1 = open('new.Survdiff.median.rfs.overlap.all.2.txt', 'wt')
#fh2 = open('Coxph.univariate.median.os.overlap.methy.txt', 'wt')
#fh3 = open('Coxph.multivariate.median.os.overlap.methy.txt', 'wt')


with open('../TCGA.Expr.univariate.survdiff.rfs.sig.symbol.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene':
            fh1.write(line+'\n')
            continue
        
        gene = lst[0].split('|')[0]
        geneSet1[gene] = line
                
with open('new.TCGA.Expr.multivariate.survdiff.rfs.sig.symbol.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene':
            continue
        
        gene = lst[0].split('|')[0]
        if gene in geneSet1.keys():
            geneSet2[gene] = line


with open('../GEO.Expr.univariate.survdiff.rfs.sig.symbol.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene':
            continue
        
        if lst[0] in geneSet2.keys():
            if lst[0] not in geneSet3.keys():
                geneSet3[lst[0]] = []
            geneSet3[lst[0]].append(line)
            

with open('../GEO.Expr.multivariate.survdiff.rfs.sig.symbol.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene':
            continue
            
        if lst[0] in geneSet3.keys():
            if lst[0] not in geneSet4.keys():
                geneSet4[lst[0]] = []
            geneSet4[lst[0]].append(line)
        


with open('../TCGA.Methy.univariate.survdiff.rfs.sig.symbol.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene':
            continue
        
        if lst[0] in geneSet4.keys():
            if lst[0] not in geneSet5.keys():
                geneSet5[lst[0]] = []
            geneSet5[lst[0]].append(line)
            
            
with open('new.TCGA.Methy.multivariate.survdiff.rfs.sig.symbol.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        if lst[0] == 'Gene':
            continue
            
        if lst[0] in geneSet4.keys():
            if lst[0] not in geneSet6.keys():
                geneSet6[lst[0]] = []
            geneSet6[lst[0]].append(line)
                
                
for gene in geneSet4.keys():
    fh1.write(geneSet1[gene]+'\n\n')
    fh1.write(geneSet2[gene]+'\n\n')
    fh1.write('\n'.join(geneSet3[gene])+'\n\n')
    fh1.write('\n'.join(geneSet4[gene])+'\n\n')
    if gene in geneSet5.keys():
        fh1.write('\n'.join(geneSet5[gene])+'\n\n')
    else:
        fh1.write(gene+'\tNA\n\n')
    if gene in geneSet6.keys():
        fh1.write('\n'.join(geneSet6[gene])+'\n\n\n\n')
    else:
        fh1.write(gene+'\tNA\n\n\n\n')      
        
        
fh1.close()

In [150]:
geneSet4.keys()

odict_keys(['C19orf40', 'L1CAM', 'TMEM159', 'SCG2', 'EFNB3', 'NOVA2', 'SSBP2', 'EMILIN1', 'ATP10A', 'CYGB', 'LMNB2', 'CXCL10', 'STMN2', 'RAI2', 'ITGB1', 'RCC1', 'SECISBP2L', 'BTBD19', 'MN1', 'SMARCA1', 'RECK', 'FGF2', 'SCN9A', 'PDE1A', 'PCDHGA10', 'ZBTB20', 'NPR2', 'FAM90A1', 'ANXA6', 'CYBRD1', 'SUSD4', 'STON1', 'CDK14', 'PSMD11', 'STAG2', 'CCNO', 'CALD1', 'FAM198B', 'TUBA8', 'BBS9', 'TYRP1', 'F8', 'SDPR', 'TIMP3', 'SGCD', 'PRKD1', 'NCL', 'E2F1', 'LMO2', 'TAGLN', 'ABCB4', 'AKAP12', 'AKT3', 'FERMT2', 'ICAM1', 'ABCA6', 'TCF7L1', 'IGFBP3', 'SLC2A3', 'GPRC5B', 'MAP1B', 'TMEM35', 'MAB21L2', 'FZD5', 'SFPQ', 'TOMM40', 'CCNF', 'TOP1', 'LAMA4', 'MLH3', 'CSGALNACT2', 'COLEC12', 'PDE3A', 'GLS', 'NOV', 'LDB2', 'PEG3', 'NAP1L3', 'EFEMP1', 'PLN', 'UST', 'GUCY1B3', 'TMEM204', 'ATP8A2', 'DDX28', 'TLR4', 'CNTNAP1', 'GLO1', 'MID2', 'KLHL20', 'TENC1', 'WIPF1', 'GLRB', 'PBXIP1', 'CLCN3', 'POP1', 'GREM1', 'C14orf132', 'HJURP', 'LIG1', 'DIO2', 'CDC37', 'TNS1', 'TACC3', 'MYLK', 'LIMS2', 'ARMCX1'])

In [155]:
geneSet6.keys()

odict_keys(['CLCN3', 'ARMCX1', 'TMEM204', 'PDE1A', 'E2F1', 'TNS1', 'MAB21L2', 'PCDHGA10', 'PRKD1', 'LAMA4', 'FZD5', 'CDC37', 'SGCD', 'AKAP12', 'DIO2', 'STAG2', 'MAP1B', 'RCC1', 'CCNF', 'FGF2', 'LDB2', 'SECISBP2L', 'SMARCA1', 'DDX28', 'EFEMP1', 'LMNB2', 'STON1', 'RAI2', 'L1CAM', 'GLRB', 'SCN9A', 'NCL', 'TCF7L1', 'SFPQ', 'ZBTB20', 'AKT3', 'CALD1', 'MID2', 'SLC2A3', 'CYBRD1', 'EFNB3', 'CYGB', 'GUCY1B3', 'ATP10A', 'TAGLN', 'RECK', 'PEG3', 'POP1', 'UST', 'ATP8A2', 'SDPR', 'SSBP2', 'BBS9', 'LIG1', 'IGFBP3', 'MYLK', 'ANXA6', 'CDK14', 'TIMP3', 'SCG2', 'STMN2', 'CXCL10', 'NOVA2', 'COLEC12', 'MN1', 'F8', 'C14orf132', 'FAM198B', 'WIPF1'])

In [36]:
########## target prediction

### organization
fh = open('Nonconserved_Site_Context_Scores.hsa.txt', 'wt')

targetscan = OrderedDict()
with open('Nonconserved_Site_Context_Scores.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0].startswith('Gene'):
            fh.write(line+'\n')
        
        
        elif lst[3] == '9606':
            if lst[4] not in targetscan.keys():
                targetscan[lst[4]] = []
                
            targetscan[lst[4]].append(line)

            
for ke in sorted(targetscan.keys()):
    fh.write('\n'.join(targetscan[ke])+'\n')
    
fh.close()
    

In [39]:

targetscan = OrderedDict()

fh = open('Nonconserved_Site_Context_Scores.hsa.clean.txt', 'wt')

with open('Nonconserved_Site_Context_Scores.hsa.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0].startswith('Gene'):
            fh.write('miRNA\tgene\n')
            continue
        
        elif lst[4] not in targetscan.keys():
            targetscan[lst[4]] = []
        
        if lst[1] not in targetscan[lst[4]]:
            fh.write(lst[4]+'\t'+lst[1]+'\n')
            targetscan[lst[4]].append(lst[1])
        

fh.close()

In [40]:
fh = open('Nonconserved_Site_Context_Scores.hsa.clean.summary.txt', 'wt')

for ke, val in targetscan.items():
    fh.write(ke+'\t'+';'.join(val)+'\n')
    
fh.close()

In [41]:
targetscan = OrderedDict()

with open('Nonconserved_Site_Context_Scores.hsa.clean.summary.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        targetscan[lst[0]] = lst[-1].split(';')

In [42]:
len(targetscan)

2603

In [52]:
mir = OrderedDict()
fh1 = open('miRNA.Survdiff.os.overlap.targets.txt', 'wt')
fh2 = open('miRNA.Survdiff.os.overlap.targets.seperate.txt', 'wt')


with open('miRNA.Survdiff.os.overlap.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0] == '':
            continue
            
        elif lst[0] not in mir.keys():
            mir[lst[0]] = 1
            
            
for ke in mir.keys():
    fh1.write(ke+'\t'+';'.join(targetscan[ke])+'\n')
    
    for val in targetscan[ke]:
        fh2.write(ke+'\t'+val+'\n')
    
fh1.close()
fh2.close()

In [53]:
ovlp = []

with open('new2.Survdiff.median.os.overlap.all.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0] == '' or lst[0] == 'Gene':
            continue
            
        elif lst[0] not in ovlp:
            ovlp.append(lst[0])

In [54]:
fh = open('Survdiff.median.os.overlap.all.miRNA.txt', 'wt')

with open('miRNA.Survdiff.os.overlap.targets.seperate.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[-1] in ovlp:
            fh.write(line+'\n')
            
fh.close()

In [55]:
mir = OrderedDict()
fh1 = open('miRNA.Survdiff.rfs.overlap.targets.txt', 'wt')
fh2 = open('miRNA.Survdiff.rfs.overlap.targets.seperate.txt', 'wt')

with open('miRNA.Survdiff.rfs.overlap.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0] == '':
            continue
            
        elif lst[0] not in mir.keys():
            mir[lst[0]] = 1
            
            
for ke in mir.keys():
    fh1.write(ke+'\t'+';'.join(targetscan[ke])+'\n')
    
    for val in targetscan[ke]:
        fh2.write(ke+'\t'+val+'\n')
    
fh1.close()
fh2.close()

In [56]:
ovlp = []

with open('new2.Survdiff.median.rfs.overlap.all.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[0] == '' or lst[0] == 'Gene':
            continue
            
        elif lst[0] not in ovlp:
            ovlp.append(lst[0])

In [57]:
fh = open('Survdiff.median.rfs.overlap.all.miRNA.txt', 'wt')

with open('miRNA.Survdiff.rfs.overlap.targets.seperate.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')
        
        if lst[-1] in ovlp:
            fh.write(line+'\n')
            
fh.close()

In [19]:
targetscan['hsa-miR-139-5p'][1:5]

['ADCYAP1R1', 'AEBP2', 'AFF3', 'AGO2']

In [28]:
len(targetscan)

329

In [14]:
targetscan = OrderedDict()

with open('summary_Predicted_Targets_Context_Scores.default_predictions_95.txt') as f:
    for line in f:
        line = line.rstrip()
        lst = line.split('\t')

        if lst[1] not in targetscan.keys():
            targetscan[lst[1]] = lst[-1].split(',')
        
        
        